# Python并发编程与扩展开发教程

## 目录

1. [CPU密集型与IO密集型场景介绍](#1-cpu密集型与io密集型场景介绍)
2. [多线程编程 (threading)](#2-多线程编程-threading)
3. [多进程编程 (multiprocessing)](#3-多进程编程-multiprocessing)
4. [异步编程 (asyncio)](#4-异步编程-asyncio)
5. [C扩展开发](#5-c扩展开发)
6. [C++扩展开发](#6-c扩展开发)
7. [Rust扩展开发](#7-rust扩展开发)
8. [性能对比与最佳实践](#8-性能对比与最佳实践)

## 1. CPU密集型与IO密集型场景介绍

### 1.1 CPU密集型场景
**示例场景**：大规模数据计算，如矩阵乘法、图像处理、加密解密等。
**特点**：程序主要受CPU资源限制，需要大量的计算资源。
**适合方案**：多进程编程、C/Rust扩展加速。

In [ ]:
# CPU密集型任务示例：斐波那契数列计算
import time
from src.cpu_intensive import fibonacci, fibonacci_iter

# 测试递归斐波那契
start_time = time.time()
result = fibonacci(35)
end_time = time.time()
print(f"递归 fib(35) = {result}, 耗时: {end_time - start_time:.4f}秒")

# 测试迭代斐波那契
start_time = time.time()
result = fibonacci_iter(35)
end_time = time.time()
print(f"迭代 fib(35) = {result}, 耗时: {end_time - start_time:.4f}秒")

### 1.2 IO密集型场景
**示例场景**：网络请求、文件读写、数据库操作等。
**特点**：程序主要等待IO操作完成，CPU利用率较低。
**适合方案**：多线程编程、异步编程。

In [ ]:
# IO密集型任务示例：HTTP请求
import time
from src.io_intensive import sync_http_requests, TEST_URLS

start_time = time.time()
sync_http_requests(TEST_URLS)
end_time = time.time()
print(f"同步执行{len(TEST_URLS)}个HTTP请求，耗时: {end_time - start_time:.4f}秒")

## 2. 多线程编程 (threading)

多线程适合IO密集型任务，通过线程切换来提高CPU利用率。
Python的GIL限制了多线程在CPU密集型任务上的性能。

In [ ]:
import threading
import time
from src.cpu_intensive import fibonacci_iter

# 测试用例：计算多个斐波那契数
test_cases = [35, 36, 37, 38]

# 同步执行
def sync_execution():
    results = []
    start_time = time.time()
    for n in test_cases:
        results.append(fibonacci_iter(n))
    end_time = time.time()
    print(f"同步执行耗时: {end_time - start_time:.4f}秒")
    return results

# 多线程执行
def thread_execution():
    results = [0] * len(test_cases)
    
    def worker(index, n):
        results[index] = fibonacci_iter(n)
    
    threads = []
    start_time = time.time()
    
    # 创建并启动线程
    for i, n in enumerate(test_cases):
        t = threading.Thread(target=worker, args=(i, n))
        threads.append(t)
        t.start()
    
    # 等待所有线程完成
    for t in threads:
        t.join()
    
    end_time = time.time()
    print(f"多线程执行耗时: {end_time - start_time:.4f}秒")
    return results

# 运行测试
print("=== 多线程 vs 同步执行测试 ===")
sync_results = sync_execution()
thread_results = thread_execution()

print(f"结果一致性: {sync_results == thread_results}")

## 3. 多进程编程 (multiprocessing)

多进程适合CPU密集型任务，通过创建多个进程绕过GIL限制。
每个进程有独立的内存空间和Python解释器。

In [ ]:
import multiprocessing
import time
from src.cpu_intensive import fibonacci_iter

# 测试用例：计算多个斐波那契数
test_cases = [35, 36, 37, 38]

# 多进程执行
def multiprocess_execution():
    start_time = time.time()
    
    # 使用进程池
    with multiprocessing.Pool(processes=multiprocessing.cpu_count()) as pool:
        results = pool.map(fibonacci_iter, test_cases)
    
    end_time = time.time()
    print(f"多进程执行耗时: {end_time - start_time:.4f}秒")
    return results

# 运行测试
print("=== 多进程执行测试 ===")
mp_results = multiprocess_execution()
print(f"结果: {mp_results}")

## 4. 异步编程 (asyncio)

异步编程适合IO密集型任务，通过事件循环实现非阻塞IO操作。
使用async/await语法，代码结构清晰。

In [ ]:
import asyncio
from src.io_intensive import async_http_requests, TEST_URLS

async def main():
    print(f"=== 异步HTTP请求测试 ===")
    time_taken = await async_http_requests(TEST_URLS)
    print(f"异步执行{len(TEST_URLS)}个HTTP请求，耗时: {time_taken:.4f}秒")

# 运行异步测试
asyncio.run(main())

## 5. C扩展开发

C扩展可以显著提高CPU密集型任务的性能。
我们已经编译了一个C扩展模块`fib_c`，用于加速斐波那契数列计算。

In [ ]:
import time
from src.cpu_intensive import fibonacci_iter as py_fib_iter

# 尝试导入C扩展
import sys
import os

# 将lib目录添加到Python路径
lib_dir = os.path.abspath('lib')
if lib_dir not in sys.path:
    sys.path.insert(0, lib_dir)

try:
    import fib_c
    has_c_extension = True
    print("✅ 成功导入C扩展模块 fib_c")
except ImportError as e:
    has_c_extension = False
    print(f"❌ 无法导入C扩展模块: {e}")

# 性能对比测试
if has_c_extension:
    test_n = 40
    
    # Python实现
    start_time = time.time()
    py_result = py_fib_iter(test_n)
    py_time = time.time() - start_time
    
    # C扩展实现
    start_time = time.time()
    c_result = fib_c.fibonacci_iter(test_n)
    c_time = time.time() - start_time
    
    print(f"=== Python vs C扩展性能对比 ===")
    print(f"计算 fib({test_n})")
    print(f"Python实现: 结果={py_result}, 耗时={py_time:.6f}秒")
    print(f"C扩展实现: 结果={c_result}, 耗时={c_time:.6f}秒")
    print(f"性能提升: {py_time/c_time:.2f}倍")

## 6. C++扩展开发

C++扩展结合了C++的高性能和Python的易用性。
我们使用pybind11库来简化C++扩展的开发。

In [ ]:
# 尝试导入C++扩展
try:
    import matrix_cpp
    has_cpp_extension = True
    print("✅ 成功导入C++扩展模块 matrix_cpp")
except ImportError as e:
    has_cpp_extension = False
    print(f"❌ 无法导入C++扩展模块: {e}")

# 使用示例
if has_cpp_extension:
    # 创建矩阵操作对象
    mo = matrix_cpp.MatrixOperations()
    
    # 测试斐波那契计算
    result = mo.fibonacci(35)
    print(f"C++扩展计算 fib(35) = {result}")
    
    # 测试快速排序
    test_list = [3.1, 1.4, 1.5, 9.2, 6.5, 3.5, 8.9]
    mo.quick_sort(test_list)
    print(f"快速排序结果: {test_list}")

## 7. Rust扩展开发

Rust扩展提供了内存安全和高性能的特性。
我们使用PyO3库来开发Rust扩展。

In [ ]:
# 尝试导入Rust扩展
try:
    import fib_rust
    has_rust_extension = True
    print("✅ 成功导入Rust扩展模块 fib_rust")
except ImportError as e:
    has_rust_extension = False
    print(f"❌ 无法导入Rust扩展模块: {e}")
    print("💡 提示: 请确保已编译Rust扩展并将其添加到Python路径")

## 8. 性能对比与最佳实践

### 8.1 不同并发模型的性能对比

In [ ]:
import time
import threading
import multiprocessing
from src.cpu_intensive import fibonacci_iter as py_fib_iter

# 测试配置
TEST_N = 38
NUM_TASKS = 4
TEST_CASES = [TEST_N] * NUM_TASKS

print(f"=== 不同并发模型性能对比测试 ===")
print(f"测试任务: 计算{NUM_TASKS}个 fib({TEST_N})")
print(f"CPU核心数: {multiprocessing.cpu_count()}")
print("-" * 60)

# 1. 同步执行
start_time = time.time()
[py_fib_iter(n) for n in TEST_CASES]
sync_time = time.time() - start_time
print(f"1. 同步执行: {sync_time:.4f}秒")

# 2. 多线程执行
def thread_worker(n):
    return py_fib_iter(n)

start_time = time.time()
threads = []
for n in TEST_CASES:
    t = threading.Thread(target=thread_worker, args=(n,))
    threads.append(t)
    t.start()
for t in threads:
    t.join()
thread_time = time.time() - start_time
print(f"2. 多线程执行: {thread_time:.4f}秒 (加速比: {sync_time/thread_time:.2f}x)")

# 3. 多进程执行
start_time = time.time()
with multiprocessing.Pool() as pool:
    pool.map(py_fib_iter, TEST_CASES)
mp_time = time.time() - start_time
print(f"3. 多进程执行: {mp_time:.4f}秒 (加速比: {sync_time/mp_time:.2f}x)")

# 4. C扩展执行
try:
    import fib_c
    start_time = time.time()
    [fib_c.fibonacci_iter(n) for n in TEST_CASES]
    c_time = time.time() - start_time
    print(f"4. C扩展同步执行: {c_time:.4f}秒 (加速比: {sync_time/c_time:.2f}x)")
    
    # 5. C扩展 + 多进程
    start_time = time.time()
    with multiprocessing.Pool() as pool:
        pool.map(fib_c.fibonacci_iter, TEST_CASES)
    c_mp_time = time.time() - start_time
    print(f"5. C扩展 + 多进程: {c_mp_time:.4f}秒 (加速比: {sync_time/c_mp_time:.2f}x)")
except ImportError:
    print("4. C扩展: 不可用")
    print("5. C扩展 + 多进程: 不可用")

print("-" * 60)
print("性能对比总结:")
print(f"  同步执行 > 多线程执行 > 多进程执行 > C扩展同步 > C扩展 + 多进程")
print("")
print("最佳实践建议:")
print("- IO密集型任务: 优先使用异步编程或多线程")
print("- CPU密集型任务: 优先使用多进程或C/Rust扩展")
print("- 极致性能需求: 结合使用C/Rust扩展 + 多进程")

### 8.2 实际应用场景建议

| 场景类型 | 推荐方案 | 适用场景 |
|---------|---------|---------|
| IO密集型 | 异步编程 (asyncio) | 网络爬虫、API服务、WebSocket应用 |
| IO密集型 | 多线程 (threading) | 文件处理、数据库操作、简单网络请求 |
| CPU密集型 | 多进程 (multiprocessing) | 数据分析、科学计算、批量处理 |
| CPU密集型 | C/Rust扩展 | 性能关键路径、算法库、数值计算 |
| 混合场景 | 多进程 + 异步 | Web服务器、复杂应用程序 |

## 9. 项目结构与代码组织

In [ ]:
# 查看项目结构
!ls -la

In [ ]:
# 查看src目录内容
!ls -la src/

## 10. 运行测试用例

In [ ]:
# 运行测试用例
!uv run pytest tests/ -v

## 总结

本教程介绍了Python并发编程的三种主要方式：
1. **多线程编程**：适合IO密集型任务，受GIL限制
2. **多进程编程**：适合CPU密集型任务，绕过GIL限制
3. **异步编程**：适合高并发IO密集型任务，单线程事件循环

同时，我们还介绍了如何使用C、C++和Rust扩展来提高Python程序的性能：
- C扩展：直接使用Python C API，性能最优
- C++扩展：使用pybind11库，开发更便捷
- Rust扩展：使用PyO3库，内存安全且高性能

在实际应用中，应根据任务类型和性能需求选择合适的并发模型和扩展方案。